In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
file_path = r'C:\Users\tonyd\OneDrive\바탕 화면\4-2\딥러닝 응용\data\Raisin_dataset.xlsx'

data = pd.read_excel(file_path)
print(data.head())

    Area  MajorAxisLength  MinorAxisLength  Eccentricity  ConvexArea  \
0  87524       442.246011       253.291155      0.819738       90546   
1  75166       406.690687       243.032436      0.801805       78789   
2  90856       442.267048       266.328318      0.798354       93717   
3  45928       286.540559       208.760042      0.684989       47336   
4  79408       352.190770       290.827533      0.564011       81463   

     Extent  Perimeter    Class  
0  0.758651   1184.040  Kecimen  
1  0.684130   1121.786  Kecimen  
2  0.637613   1208.575  Kecimen  
3  0.699599    844.162  Kecimen  
4  0.792772   1073.251  Kecimen  


In [4]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

# Class 열을 Label Encoding
data['Class'] = label_encoder.fit_transform(data['Class'])

# 변환된 Class 값 확인
print(data['Class'].value_counts())

Class
1    450
0    450
Name: count, dtype: int64


In [5]:
# 특징과 레이블 분리 (예시로 마지막 열을 레이블로 가정)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [6]:
# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
# PCA로 차원 축소 (예: 95% 설명 분산을 유지하는 주성분 선택)
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)


In [8]:
# 학습용 및 테스트용 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [9]:
# 딥러닝 분류 모델 생성
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_pca.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # 이진 분류의 경우, 다중 클래스는 'softmax' 사용

c:\Users\tonyd\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # 다중 클래스의 경우 'categorical_crossentropy' 사용


In [11]:
# 모델 학습
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.4404 - loss: 0.7311 - val_accuracy: 0.8750 - val_loss: 0.5251
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8229 - loss: 0.5170 - val_accuracy: 0.8958 - val_loss: 0.3934
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8190 - loss: 0.4410 - val_accuracy: 0.9028 - val_loss: 0.3307
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8737 - loss: 0.3509 - val_accuracy: 0.8958 - val_loss: 0.3004
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8656 - loss: 0.3441 - val_accuracy: 0.9028 - val_loss: 0.2938
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8784 - loss: 0.3373 - val_accuracy: 0.9028 - val_loss: 0.2883
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8591 - loss: 0.3661 - val_accuracy: 0.9028 - val_loss: 0.2861
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8660 - loss: 0.3443 - val_accuracy: 0.9028 - val_loss

In [12]:
# 예측 및 평가
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [13]:
# 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f'Model accuracy: {accuracy * 100:.2f}%')

Model accuracy: 86.67%
